In [ ]:
import pandas as pd
import nltk
import re
import numpy as np
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from nltk.corpus import stopwords
import pickle

In [ ]:
def split_df(df, column_name): 
    """Split Sentences DF by the year column so that we can do the LR by year"""

    #get all unique values in col
    col_vals = df[column_name].unique()

    split = [df[df[column_name] == value] for value in col_vals]
    
    #Return the list of split dataframes
    return split

#use the new function to split the original sentences df 
splitted_df = split_df(sentences_df, 'year')

#print each of the DFs
for df_year_data in splitted_df:
    display(df_year_data)

**Binary Logistic Regression**

In [ ]:
#vectorizer = TfidfVectorizer(max_features= 1000, lowercase=False, tokenizer=False)
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

#X_train = tfidf.fit_transform(X_train)
#X_test = tfidf.fit_transform(X_test)

In [ ]:
#GridSearchCV Packages - this still needs changing and fitting into the LR function!
from sklearn.model_selection import GridSearchCV
import warnings 
warnings.filterwarnings("ignore")


# parameter grid
parameters = {"penalty": ['l1','l2'], 
              "C": np.logspace(-3,3,7),
              "solver": ['newton-cg', 'lbfgs', 'liblinear'],
}

#GridSearchCV
logreg = LogisticRegression()
clf_logreg = GridSearchCV(logreg, 
                          param_grid = parameters, 
                          scoring = "accuracy", 
                          cv = 10)

clf_logreg.fit(X_train, y_train)

**Logistic Regression Classifier by Year**

In [ ]:
def logistic_regression_year(df, target_col, text_col, year_col):
    years = df[year_col].unique()
    results = {}

    #split data 
    for year in years:
        df_year = df.loc[df[year_col]==year].copy()
        df_year['text'] = df_year[text_col].apply(lambda x: ' '.join(map(str, x)))
        X = df_year[text_col].apply(lambda x: str(x))
        y = df_year[target_col]

    #train test split
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        tfidf = TfidfVectorizer()
        X_train_tfidf = tfidf.fit_transform(X_train)
        X_test_tfidf = tfidf.transform(X_test)

    #run the classifier 
        clf = LogisticRegression()
        clf.fit(X_train_tfidf, y_train)
        y_pred = clf.predict(X_test_tfidf)

    #performance 
        accuracy = clf.score(X_test_tfidf, y_test)
        class_report = classification_report(y_test, y_pred, zero_division = 0)
        results[year] = {'accuracy': accuracy, 'classification_report': class_report}
        print(f"Year: {year}")
        print(f"Accuracy: {accuracy}")
        print(f"Classification Report:\n{class_report}")

        #coefficients
        coefs = clf.coef_[0]
        sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
        high_coef = sorted_coef[:5]
        low_coef = sorted_coef[-5:]
        

        #print coefficient results 
        print(f"\n Highest coefs:")
        for i in high_coef: 
            print(i)
        
        print(f"\n Lowest coefs:")
        for i in low_coef: 
            print(i)
    

In [ ]:
logistic_regression_year(sentences_df, 'col_type', 'encoded_sentences', 'year')